In [1]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from config import db_password
import json
import re

In [2]:
# Get the three data files to be used in the function

In [3]:
# Function to get json data and check to make sure there were no problems importing.
# **Note file_dir path will need to be changed for different users**
def get_wiki_data():
    try:
        
        file_dir = "C:/Users/16136/Desktop/DA Class Folder/Git/Movies-ETL"
        with open (f'{file_dir}/Resources/wikipedia.movies.json', mode='r')as file:
            wiki_movies_raw = json.load(file)
        print(f"Successfully loaded file.")
    except:
        print("Error loading file.")
   
    
    return wiki_movies_raw

In [4]:
wiki_movies_raw = get_wiki_data()
len(wiki_movies_raw)

Successfully loaded file.


7311

In [5]:
# Function to get the kagggle meta_data
def get_kaggle_data():
    try:
        file_path_kaggle_metadata = os.path.join('Resources', 'movies_metadata.csv')
        kaggle_metadata = pd.read_csv(file_path_kaggle_metadata, low_memory=False)
        print(f"Kaggle data successfully loaded.")
        return kaggle_metadata 
    except:
        print(f"Error occurred importing Kaggle Metadata.")

In [6]:
kaggle_metadata = get_kaggle_data()
kaggle_metadata.head(1)

Kaggle data successfully loaded.


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [7]:
# Function to get the ratings data
def get_ratings_data():
    try:
        file_path_ratings = os.path.join('Resources', 'ratings.csv')
        ratings = pd.read_csv(file_path_ratings, low_memory=False)
        print(f"Ratings data successfully imported.")
        return ratings
    except:
        print(f"Error occurred importing Kaggle data.")

In [8]:
ratings = get_ratings_data()
ratings.head(1)

Ratings data successfully imported.


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529


In [47]:
def etl_wiki_kaggle_ratings(wiki_movies_raw, kaggle_metadata, ratings):
    
    # Function to get json data and check to make sure there were no problems importing.
    # **Note file_dir path will need to be changed for different users**
    def get_wiki_data():
        
        try:
            if len(wiki_movies_raw) == 7311:
                wiki_movies = [movie for movie in wiki_movies_raw
                if ('Director' in movie or 'Directed by' in movie)
                and 'imdb_link' in movie
                and 'No. of episodes' not in movie]
                print(f"Wikipedia movies json data matches specified requirements.")
                
            else:
                print("Error importing json data. Please try again.")
        except:
            print(f"Error occurred after file was loaded")
        return wiki_movies
    
    wiki_movies = get_wiki_data()
    #print(wiki_movies)

    
    
    def clean_movie(movie):
        
        try:
            #remember movie is local and will not affect the global movie variable
            movie = dict(movie)
            alt_titles = {}
            for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                        'Hangul','Hebrew','Hepburn','Japanese','Literally',
                        'Mandarin','McCune–Reischauer','Original title','Polish',
                        'Revised Romanization','Romanized','Russian',
                        'Simplified','Traditional','Yiddish']:
                
                if key in movie:
                    alt_titles[key] = movie[key]
                    movie.pop(key)
            
            if len(alt_titles) > 0:
                movie['alt_titles'] = alt_titles

             # merge column names
            def change_column_name(old_name, new_name):
                if old_name in movie:
                    movie[new_name] = movie.pop(old_name)
            
            change_column_name('Adaptation by', 'Writer(s)')
            change_column_name('Country of origin', 'Country')
            change_column_name('Directed by', 'Director')
            change_column_name('Distributed by', 'Distributor')
            change_column_name('Edited by', 'Editor(s)')
            change_column_name('Length', 'Running time')
            change_column_name('Original release', 'Release date')
            change_column_name('Music by', 'Composer(s)')
            change_column_name('Produced by', 'Producer(s)')
            change_column_name('Producer', 'Producer(s)')
            change_column_name('Productioncompanies ', 'Production company(s)')
            change_column_name('Productioncompany ', 'Production company(s)')
            change_column_name('Released', 'Release Date')
            change_column_name('Release Date', 'Release date')
            change_column_name('Screen story by', 'Writer(s)')
            change_column_name('Screenplay by', 'Writer(s)')
            change_column_name('Story by', 'Writer(s)')
            change_column_name('Theme music composer', 'Composer(s)')
            change_column_name('Written by', 'Writer(s)')            
            
            return(movie)
        except:
            print(f"Error changing column names." )
    try:        
        # Clean movies
        clean_movies = [clean_movie(movie) for movie in wiki_movies]
        wiki_movies_df = pd.DataFrame(clean_movies)
        # Let the user know what's happening
        print("Wikipedia DataFrame has been created.")
        #print(wiki_movies_df)
    except:
        print("DataFrame not created")
    
    # Drop duplicate IMDB ID's
    wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
    #print(len(wiki_movies_df))
    wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    #print(len(wiki_movies_df))
    
    # Remove mostly null columns
    #print(len(wiki_movies_df.columns.tolist()))
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    #print(len(wiki_movies_df.columns.tolist()))
    
    # Clean box office column
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
    
    # Drop na
    box_office = wiki_movies_df['Box office'].dropna()
    
    # Convert lists to strings
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    # Find values given as range
    box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    
    def parse_dollars(s):
        try:
            # if s is not a string, return NaN
            if type(s) != str:
                return np.nan

            # if input is of the form $###.# million
            if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

                # remove dollar sign and " million"
                s = re.sub('\$|\s|[a-zA-Z]','', s)

                # convert to float and multiply by a million
                value = float(s) * 10**6

                # return value
                return value

            # if input is of the form $###.# billion
            elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

                # remove dollar sign and " billion"
                s = re.sub('\$|\s|[a-zA-Z]','', s)

                # convert to float and multiply by a billion
                value = float(s) * 10**9

                # return value
                return value

            # if input is of the form $###,###,###
            elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

                # remove dollar sign and commas
                s = re.sub('\$|,','', s)

                # convert to float
                value = float(s)

                # return value
                return value

            # otherwise, return NaN
            else:
                return np.nan
        
        except: 
            
            print(f"Something has gone wrong with applying regx to box_office values.")
            
    try:        
        # Change formatting for values in box office to new column 'box_0ffice' and drop old column 'Box office'
        wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

        wiki_movies_df.drop('Box office', axis=1, inplace=True)
        #print(wiki_movies_df.columns.tolist())
    
    except:
        print(f"Error assigning 'box_office' values or dropping 'Box_office' column.")
    
    # Clean wiki budget column
    try:
        # Drop na
        budget = wiki_movies_df['Budget'].dropna()
       
        # Convert any lists to strings
        budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
        
        # Remove any values between dollar sign and hyphen
        budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
        
        # Remove citation references
        budget = budget.str.replace(r'\[\d+\]\s*', '')
        
        # Assign values extrated with regex to new column
        wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
        
        # Drop the original budget column
        wiki_movies_df.drop('Budget', axis=1, inplace=True)
    except:
        print(f"Error encountered while cleaning 'budget' data.")
    
    # Clean release date data:
    try:
        # Drop na values in release date column
        release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

        # Apply regex forms
        # Release date regex forms
        date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
        date_form_two = r'\d{4}.[01]\d.[123]\d'
        date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
        date_form_four = r'\d{4}'
        
        # Use forms and change datatype to datetime 
        wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    
    except:
        print(f"Error cleaning release date data")
        
    try:
        # Clean Running times data:
        # Drop na
        running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
        
        # Apply regex
        running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
        
        # Convert empty strings to nan with 'coerce' then to 0 with fillna(0)
        running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
        
        #function that will convert the hour capture groups and minute capture groups to minutes if the pure minutes capture group is zero
        wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
        
        # Drop Running time column
        wiki_movies_df.drop('Running time', axis=1, inplace=True)
    except:
        print(f"Error cleaning running_time data")
    print("Wikipedia DataFrame has been cleaned.")
    
    ### Kaggle Data ###
    
    try:
        # Drop adult column from hackathon data
        kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
        print(f"Successfully dropped Kaggle 'adult' column.")
    except:
        print(f"Error dropping adult column from kaggle_metadata")
        
    try:
        # create a column with boolean masks
        kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
        
        # Change columns data type
        kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
        kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
        kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
        kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])
        print(f"Successfully cleaned Kaggle data.")
    except:
        print(f"Issue modifying Kaggle data types.")
    
    ### Ratings Data ###
    
    # Convert ratings timestamp to datetime
    try:
        ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
    except:
        print(f"Error converting ratings timestamp to datetime")
    
    ### Merge wiki_movies_df and kaggle_metadata ###
    
    try:
        movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])
        print(f"Successfully merged wiki_movies_df and kaggle_metadata.")
    except:
        print("Error merging wiki_movies_df and kaggle_metadata.")
        
    try:
        # Drop lofi dupplicate columns
        movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)
        
        #  Fill missing kaggle data with wiki data
        def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
            df[kaggle_column] = df.apply(lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column], axis=1)
            df.drop(columns=wiki_column, inplace=True)
            
        fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
        fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
        fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
        print(f"Successfully filled missing Kaggle data with wiki data.")
        
    except:
        print(f"Error filling missing Kaggle data.")
    
    try:
        # Reorder columns
        movies_df = movies_df[['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                           'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                           'genres','original_language','overview','spoken_languages','Country',
                           'production_companies','production_countries','Distributor',
                           'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                          ]]
        print(f"Successfully reordered columns in movies_df.")
    except:
        print(f"Error re-ordering columns.")
    
    try:
        # Rename columns
        movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)
        print(f"Successfully renamed columns.")
    
    except:
        print(f"Error renaming columns")
    
    ### Transform Rating Data and merge with movies_df ###
    try:
        # pivot data so movieId is the index, rating values are the columns and rows are the count for each rating value.
        rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count().rename({'userId':'count'}, axis=1).pivot(index='movieId',columns='rating', values='count')
        
        # Rename columns with list comprehension.
        rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
        
        # Merge data with movies_df.
        movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
        
        # Fill na values with 0.
        movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
        
        print(f"Successfully merged ratings data and movies_df")
    except:
        print(f"Error transforming ratings data.")
        
    

In [48]:
etl_wiki_kaggle_ratings(wiki_movies_raw, kaggle_metadata, ratings)

Wikipedia movies json data matches specified requirements.
Wikipedia DataFrame has been created.
Wikipedia DataFrame has been cleaned.
Successfully dropped Kaggle 'adult' column.
Successfully cleaned Kaggle data.
Successfully merged wiki_movies_df and kaggle_metadata.
Successfully filled missing Kaggle data with wiki data.
Successfully reordered columns in movies_df.
Successfully renamed columns.
Successfully merged ratings data and movies_df
